# 다목적 최적화 알고리즘 성능 비교 실험

**논문용 실험**: NSGA-II, NSGA-III, SPEA2, ε-MOEA 알고리즘의 성능을 비교합니다.

## 실험 설계
- **알고리즘**: NSGA-II, NSGA-III, SPEA2, ε-MOEA
- **반복 횟수**: 각 알고리즘당 10회 독립 실행
- **세대 수**: 100 generations
- **평가 지표**:
  - **Hypervolume**: 수렴 품질 (값이 클수록 좋음)
  - **Spacing**: 해의 분포 균일성 (값이 작을수록 좋음)
  - **Diversity**: 해의 다양성 (값이 클수록 좋음)
  - **Convergence**: 수렴 속도 (값이 클수록 좋음)
  - **Execution Time**: 실행 시간 (초)

## 출력 파일
- `optimization_comparison_results.xlsx`: 원본 성능 데이터 (통계 분석용)

In [1]:
# 필수 라이브러리 임포트
import pandas as pd
import numpy as np
from load_data import load_and_process_data, create_nutrient_constraints, load_all_menus
from evaluation_function import calculate_harmony_matrix

from performance_metrics import PerformanceEvaluator
from nsga2_optimizer import NSGA2Optimizer
from nsga3_optimizer import NSGA3Optimizer
from spea2_optimizer import SPEA2Optimizer
from emoea_optimizer import EpsilonMOEAOptimizer

In [2]:
# 데이터 로딩 함수
def load_data(diet_db_path, menu_db_path, ingre_db_path):
    """
    식단 최적화에 필요한 모든 데이터 로드
    
    Returns:
        diet_db: 전체 식단 데이터베이스
        nutrient_constraints: 영양소 제약 조건
        harmony_matrix: 메뉴 간 조화도 행렬
        all_menus: 전체 메뉴 리스트
    """
    print("데이터 로딩 중...")
    diet_db = load_and_process_data(diet_db_path, menu_db_path, ingre_db_path)
    nutrient_constraints = create_nutrient_constraints()
    harmony_matrix, menus, menu_counts, _ = calculate_harmony_matrix(diet_db)
    all_menus = load_all_menus(menu_db_path, ingre_db_path)
    print(f"✅ 데이터 로딩 완료: {len(all_menus)}개 메뉴\n")
    
    return diet_db, nutrient_constraints, harmony_matrix, all_menus

def compare_optimizers_performance(initial_diet_path, diet_db, menu_db_path, ingre_db_path,
                                   all_menus, nutrient_constraints, harmony_matrix,
                                   generations: int = 100, num_runs: int = 10):
    """
    논문용 알고리즘 성능 비교 실험
    
    4가지 알고리즘(NSGA-II, NSGA-III, SPEA2, ε-MOEA)을 각각 num_runs회 반복 실행하여
    Hypervolume, Spacing, Diversity, Convergence, Execution Time을 측정합니다.
    
    Args:
        generations: 각 실행당 세대 수 (default: 100)
        num_runs: 각 알고리즘당 반복 실행 횟수 (default: 10)
    
    Returns:
        performance_results: 각 알고리즘의 성능 측정 결과 (딕셔너리)
    """
    # 초기 식단 로드 (Weekly_diet_ex.xlsx - 7일 샘플 식단)
    print("초기 식단 로딩 중...")
    weekly_diet = load_and_process_data(diet_db_path=initial_diet_path,
                                       menu_db_path=menu_db_path,
                                       ingre_db_path=ingre_db_path)
    
    print(f"✅ 초기 식단: {len(weekly_diet.meals)}끼 ({len(weekly_diet.meals)//3}일)\n")
    
    # 최적화 알고리즘 초기화
    optimizers = {
        'NSGA-II': NSGA2Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'NSGA-III': NSGA3Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'SPEA2': SPEA2Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'ε-MOEA': EpsilonMOEAOptimizer(all_menus, nutrient_constraints, harmony_matrix)
    }
    
    # 실험 정보 출력
    print("\n" + "="*80)
    print("🔬 알고리즘 성능 비교 실험 시작")
    print("="*80)
    print(f"📊 실험 설계:")
    print(f"   - 알고리즘: NSGA-II, NSGA-III, SPEA2, ε-MOEA")
    print(f"   - 반복 횟수: {num_runs}회/알고리즘")
    print(f"   - 세대 수: {generations} generations")
    print(f"   - 총 실행 횟수: {4 * num_runs}회 (4 알고리즘 × {num_runs}회)")
    print(f"\n📈 평가 지표:")
    print(f"   - Hypervolume (수렴 품질)")
    print(f"   - Spacing (분포 균일성)")
    print(f"   - Diversity (해 다양성)")
    print(f"   - Convergence (수렴 속도)")
    print(f"   - Execution Time (실행 시간)")
    print("="*80 + "\n")
    
    # 성능 평가자 초기화
    evaluator = PerformanceEvaluator(diet_db, weekly_diet, optimizers)
    performance_results = {}
    
    # 각 알고리즘을 num_runs회씩 실행하여 성능 측정
    for name in optimizers.keys():
        print(f"\n{'='*80}")
        print(f"🚀 {name} 알고리즘 실행 중 ({num_runs}회 반복)...")
        print(f"{'='*80}")
        
        performance_results[name] = evaluator.run_single_optimizer(
            optimizer_name=name,
            generations=generations,
            num_runs=num_runs
        )
        
        # 중간 결과 요약 출력
        print(f"\n✅ {name} 완료:")
        if performance_results[name]['hypervolume']:
            print(f"   평균 Hypervolume: {np.mean(performance_results[name]['hypervolume']):.4f}")
            print(f"   평균 실행시간: {np.mean(performance_results[name]['execution_time']):.2f}초")
    
    # 결과 저장
    print(f"\n{'='*80}")
    print("💾 결과 저장 중...")
    evaluator.save_combined_results_to_excel(
        performance_results,
        filename='optimization_comparison_results.xlsx'
    )
    
    print("\n" + "="*80)
    print("✅ 실험 완료!")
    print("="*80)
    print(f"📊 결과 파일: optimization_comparison_results.xlsx")
    print(f"\n다음 단계: '실험 끝났어요!' 라고 말씀하시면 자동으로")
    print(f"  1) 6개 그림 생성 (Radar, Hypervolume, Spacing, Diversity, Time, Heatmap)")
    print(f"  2) Results 섹션 초안 작성")
    print(f"  3) GitHub 업로드")
    print(f"가 진행됩니다.")
    print("="*80 + "\n")
    
    return performance_results

In [3]:
# 실험 실행
# jeongseong 데이터셋 사용
name = 'jeongseong'

diet_db_path = f'../data/sarang_DB/processed_DB/DIET_{name}.xlsx'
menu_db_path = f'../data/sarang_DB/processed_DB/Menu_ingredient_nutrient_{name}.xlsx'
ingre_db_path = f'../data/sarang_DB/processed_DB/Ingredient_Price_{name}.xlsx'

# 데이터 로드
diet_db, nutrient_constraints, harmony_matrix, all_menus = load_data(
    diet_db_path, menu_db_path, ingre_db_path
)

# 초기 식단 경로 (7일 샘플 식단 사용)
initial_diet_path = '../data/sarang_DB/processed_DB/Weekly_diet_ex.xlsx'

# 알고리즘 성능 비교 실험 실행
performance_results = compare_optimizers_performance(
    initial_diet_path,
    diet_db, menu_db_path, ingre_db_path,
    all_menus, nutrient_constraints, harmony_matrix,
    generations=100,  # 세대 수
    num_runs=10       # 반복 횟수
)

데이터 로딩 중...
✅ 데이터 로딩 완료: 931개 메뉴

초기 식단 로딩 중...
Info: Menu mappings for Day 2 저녁: ['애호박볶음 → 새송이버섯애호박볶음']
✅ 초기 식단: 21끼 (7일)


🔬 알고리즘 성능 비교 실험 시작
📊 실험 설계:
   - 알고리즘: NSGA-II, NSGA-III, SPEA2, ε-MOEA
   - 반복 횟수: 10회/알고리즘
   - 세대 수: 100 generations
   - 총 실행 횟수: 40회 (4 알고리즘 × 10회)

📈 평가 지표:
   - Hypervolume (수렴 품질)
   - Spacing (분포 균일성)
   - Diversity (해 다양성)
   - Convergence (수렴 속도)
   - Execution Time (실행 시간)


🚀 NSGA-II 알고리즘 실행 중 (10회 반복)...

Evaluating NSGA-II...
=== Generation 1/100 ===
제약조건 만족 해: 150/150
현재 세대 수집된 해: 10개
총 수집된 좋은 해: 10개
백업 해: 10개
제약조건 해: 150개
Termination condition met at generation 0

=== get_final_solutions 시작 ===
good_solutions_archive: 10개
backup_solutions: 10개
constraint_solutions: 150개
Run 1/10 completed
=== Generation 1/100 ===
제약조건 만족 해: 150/150
현재 세대 수집된 해: 5개
총 수집된 좋은 해: 5개
백업 해: 5개
제약조건 해: 150개
Termination condition met at generation 0

=== get_final_solutions 시작 ===
good_solutions_archive: 5개
backup_solutions: 5개
constraint_solutions: 150개
Run 2/10 complet